In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc

import csv

import math

In [ ]:
background_vagabond_colors = ["#fffdfa", "#fef8f0", "#fbe4cb", "#e8c29d", 
                              "#9a7b56", "#2b322e", "#f0f2f1"]
colors = ["#cec5c1", "#9f8f7f", "#924a42", "#5a2028",
        "#a4b9b1", "#4ea7a6", "#026d73", "#06494f"]
gmk_ursa = ["#A9907E", "#F3DEBA", "#ABC4AA", "#675D50"]

plt.rcParams['savefig.dpi'] = 300;  

rc('font', **{'family':'serif', 'serif':['Computer Modern']})
rc('text', usetex = True)

In [ ]:
# path = '/Users/jacksonpowell/Documents/GitHub/learningML/data_files/Gait_cycles_1/test2_SUB01_off_walk_CoM.csv'; 
example_path = '/Users/jacksonpowell/Documents/GitHub/learningML/data_files/PD-gait_2/GaCo09_02.txt';  
demo_path = '/Users/jacksonpowell/Documents/GitHub/learningML/data_files/PD-gait_2/demographics.csv'; 


In [ ]:
demo_df = pd.read_csv(demo_path)
demo_df.head()

demo_df.set_index('ID', inplace = True)
weight = demo_df.loc['GaCo01', 'Weight']

In [ ]:
headers = ["time", "L_VGRF1", "L_VGRF2", "L_VGRF3", "L_VGRF4",
           "L_VGRF5", "L_VGRF6", "L_VGRF7", "L_VGRF8", 
           "R_VGRF1", "R_VGRF2", "R_VGRF3", "R_VGRF4",
           "R_VGRF5", "R_VGRF6", "R_VGRF7", "R_VGRF8",
           "L_Total", "R_Total"]

example_df = pd.read_csv(example_path, names = headers, sep = '\t')
example_df.head()

In [ ]:
frequency = 100; 
window_duration = 2; 

start_time = 0; 
start_time_index = start_time * frequency; 
window_duration_index = window_duration * frequency; 

x = example_df.loc[:,'time']; 
y1 = example_df.loc[:,'L_Total']; 
y2 = example_df.loc[:,'R_Total']; 

x = x[start_time_index : start_time_index + window_duration_index]; 
y1 = y1[start_time_index : start_time_index + window_duration_index]; 
y2 = y2[start_time_index : start_time_index + window_duration_index]; 

# plt.plot(x, y1, color = gmk_ursa[2], linewidth = 2)
# plt.plot(x, y2, color = gmk_ursa[0], linewidth = 2)

dx = 1 / 100 # frequency essentially
dydx = 0.1 * np.gradient(y1, dx)

# plt.plot(x, dydx)

# plt.plot(x, (x * 0 + weight * 9.8), color = 'black')

# plt.title('GRF in PD walking')
# plt.xlabel('Time (s)')
# plt.ylabel('Force (N)')



In [ ]:
import glob
import os

In [ ]:
def df_process(data_frame, all_files):

    for file in all_files:
        patient = (os.path.basename(file))[:6].strip()
        patient_trial = (os.path.basename(file)).replace('.txt', ''); 

        temp_df = pd.read_csv(file, names = headers, sep = '\t'); 

        temp_L_force_df = pd.DataFrame(temp_df.loc[:,'L_Total']); 

        patient_weight = demo_df.loc[patient, 'Weight']; 

        if patient_weight > 0:
            normalized_force = 9.8 * patient_weight; 

            temp_L_force_df = temp_L_force_df / normalized_force; 

            temp_L_force_df.columns = [patient_trial]; 

            data_frame = pd.concat([data_frame, temp_L_force_df], axis = 1); 
    
    data_frame = data_frame.drop(data_frame.columns[0], axis = 1)

    return data_frame; 

In [ ]:
folder_path = '/Users/jacksonpowell/Documents/GitHub/learningML/data_files/PD-gait_2/'; 

all_files = glob.glob(os.path.join(folder_path, "GaCo**_01.txt"))

df = pd.DataFrame({1}); 

df = df_process(df, all_files); 

print(df.shape)

df_short = df.iloc[4000:5000]
df_short.columns = df.keys()

print(df_short.shape)

# df_short.plot(subplots=True)

In [ ]:
y = df_short["GaCo10_01"]
y = y[0:120]
x = np.linspace(0, 1, len(y))

dx = 1/100

dydx = np.gradient(y, x)

# print(dydx.max(), dydx.min())

cycles = []; 
start_pos = 0; 
end_pos = 0; 
i = 0; 
while i < len(dydx): 
    if dydx[i] > 5: 
        start_pos = i; 
        
        for i in range(start_pos, len(dydx), 1):
            if all(-0.1 < dydx[i:i+5]) and all(dydx[i:i+5] < 0.1): 
                end_pos = i; 
                print(dydx[i:i+5])
                print(start_pos, dydx[start_pos], end_pos, dydx[end_pos])
                break; 
    i += 1; 

start_pos = start_pos - 2; 

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 4)); 

# ax1.plot(x, y); 
# ax2.plot(x, dydx); 

In [ ]:
# plt.plot(x, y)
# plt.plot(x * 0 + 0.17, y)
# plt.plot(x * 0 + 0.83, y)

In [ ]:

# y = df_short['GaCo10_01']; 
# plt.figure(figsize=(30, 2))
# plt.plot(y)

In [ ]:
def get_step_indexes(data, sampling_rate):

    step_threshold = 0.05; 

    step_indexes = np.zeros((1000, 2)); 

    num_steps = 0; 
    step = 0; 
    i = 0; 
    while i < len(data):
        start_pos = 0; 
        end_pos = 0; 
        if data[i] > 0.1: 
            start_pos = i; 
            while data[i] > step_threshold: 
                i += 1; 
                if i == len(data):
                    step_indexes = np.delete(step_indexes, -1, 0); 
                    break; 
                elif data[i] < step_threshold:
                    end_pos = i; 
                    step_indexes[step, 0] = start_pos - 2; 
                    step_indexes[step, 1] = end_pos + 2; 
                    step += 1; 
                    num_steps += 1; 
                    if i + 10 < len(data):
                        i += 10; 
        i += 1; 
    
    step_indexes = step_indexes[:num_steps]; 

    return step_indexes, num_steps; 

In [ ]:
def average_step_duration(steps_indexes, sampling_rate):

    durations = np.zeros(len(steps_indexes)); 
    counter = 0; 

    for coords in steps_indexes: 
        durations[counter] = (coords[1] - coords[0]); 
        counter += 1; 

    average_duration = np.mean(durations); 
    std_dev = np.std(durations);  

    return durations, average_duration, std_dev; 

In [ ]:
def extract_steps(data, step_indexes, sampling_rate):

    durations, average_duration, std_dev = average_step_duration(step_indexes, sampling_rate); 

    mid_points = np.zeros(len(step_indexes)); 

    faults_list = []; 
    dist = 0; 
    counter = 0; 
    for coords in step_indexes:
        new_dist = coords[1] - coords[0]; 
        mid_points[counter] = int((coords[1] + coords[0]) / 2); 
        if new_dist > dist and new_dist < 1.5 * average_duration: 
            dist = int(new_dist); 
        elif new_dist > 1.5 * average_duration:
            faults_list.append(counter); 
        counter += 1; 
    
    step_indexes = np.delete(step_indexes, faults_list, 0); 
    mid_points = np.delete(mid_points, faults_list, 0); 

    half_dist = int(dist / 2); 

    steps = np.zeros((len(step_indexes), 2 * half_dist)); 

    clipped_list = []; 
    counter = int(0); 
    for midpoint in mid_points:
        if midpoint - half_dist > 0 and midpoint + half_dist < len(data):
            steps[counter] = data[int(midpoint - half_dist) : int(midpoint + half_dist)];
        else: 
            clipped_list.append(counter); 
            # print("NOTE: There is some clipping in the data.", counter, int(midpoint - half_dist), int(midpoint + half_dist));
        counter += 1; 

    steps = np.delete(steps, clipped_list, 0); 

    average_step = np.zeros(2 * half_dist); 
    std_dev_step = np.zeros(2 * half_dist); 
    for i in range(0, len(average_step), 1):
        average_step[i] = np.mean(steps[:,i]); 
        std_dev_step[i] = np.std(steps[:,i]); 

    return steps, average_step, std_dev_step; 

In [ ]:
def plot_average_step(data, patient, sampling_rate):
    step_indexes, num_steps = get_step_indexes(data, sampling_rate); 

    steps, average_step, std_dev_step = extract_steps(data, step_indexes, sampling_rate); 

    counter = 0; 
    x = np.linspace(0, len(steps[0]), len(steps[0])) * 0.01; 

    for step in steps:
        plt.plot(x, step, linewidth = 0.75, color = colors[counter % len(colors)]); 
        counter += 1; 

    # plt.plot(x, average_step, linewidth = 2, color = 'black', label = "Average Step")
    # plt.plot(x, average_step + std_dev_step, linewidth = 1, color = 'black', linestyle = 'dashed', label = 'Std Dev')
    # plt.plot(x, average_step - std_dev_step, linewidth = 1, color = 'black', linestyle = 'dashed')

    # plt.title(f"{patient}, $N$ = {len(steps)} Steps")
    # plt.xlabel("Time (s)")
    # plt.ylabel("Force (Body Weights)")
    # plt.legend(); 
    # plt.show(); 


In [ ]:
# for patient in df: 
#     plot_average_step(df[patient], patient, 1); 

# patient = "GaCo22_01"

# plot_average_step(df[patient][0:9000], patient, 1); 

In [ ]:
rows = int(len(df.columns) / 3); 
fig, axs = plt.subplots(rows, 3); 

fig.set_figheight(1.5 * rows); 
fig.set_figwidth(9); 

sampling_rate = 1; 
patient_counter = 0; 
for patient in df: 

    step_indexes, num_steps = get_step_indexes(df[patient], sampling_rate); 
    
    steps, average_step, std_dev_step = extract_steps(df[patient], step_indexes, sampling_rate); 

    x = np.linspace(0, len(steps[0]), len(steps[0])) * 0.01; 

    axs = axs.flatten()

    counter = 0; 
    for step in steps:
        axs[patient_counter].plot(x, step, linewidth = 0.75, color = colors[counter % len(colors)]); 
        counter += 1; 

    axs[patient_counter].plot(x, average_step, linewidth = 1, color = 'black', label = "Average Step")
    # axs[patient_counter].plot(x, average_step + std_dev_step, linewidth = 1, color = 'black', linestyle = 'dashed', label = 'Std Dev')
    # axs[patient_counter].plot(x, average_step - std_dev_step, linewidth = 1, color = 'black', linestyle = 'dashed')

    axs[patient_counter].set_title(f"{patient}, $N$ = {len(steps)}", fontsize = 10, y = 1.05, pad = 0); 
    axs[patient_counter].set_xticks([]); 
    axs[patient_counter].set_yticks([0, 1, 2]); 
    axs[patient_counter].set_ylim(0, 2); 

    # axs[patient_counter].set_xlabel("Time (s)")
    # axs[patient_counter].set_ylabel("Force (Body Weights)")
    patient_counter += 1; 
 
# plt.tight_layout()
plt.subplots_adjust(left = None, bottom = None, right = None, top = None, wspace = None, hspace = 0.3); 
plt.savefig('../images/array.png'); 
plt.show(); 


In [ ]:
folder_path_PD = '/Users/jacksonpowell/Documents/GitHub/learningML/data_files/PD-gait_2/'; 

all_files = glob.glob(os.path.join(folder_path_PD, "GaPt**_01.txt"))

df_PD = pd.DataFrame({1}); 

df_PD = df_process(df_PD, all_files); 



To do: 
* left and right foot
* compare left and right foot
* we could analyze at individual probes too...


UGHHHHHHHH SO MUCH DATA

In [ ]:
rows = int(len(df_PD.columns) / 3); 
fig, axs = plt.subplots(rows, 3); 

fig.set_figheight(1.5 * rows); 
fig.set_figwidth(8); 

sampling_rate = 1; 
patient_counter = 0; 

for patient in df_PD: 

    step_indexes, num_steps = get_step_indexes(df_PD[patient], sampling_rate); 
    
    steps, average_step, std_dev_step = extract_steps(df_PD[patient], step_indexes, sampling_rate); 

    x = np.linspace(0, len(steps[0]), len(steps[0])) * 0.01; 

    axs = axs.flatten()

    counter = 0; 
    for step in steps:
        axs[patient_counter].plot(x, step, linewidth = 0.75, color = colors[counter % len(colors)]); 
        counter += 1; 
    
    axs[patient_counter].plot(x, average_step, linewidth = 1, color = 'black', label = "Average Step")
    # axs[patient_counter].plot(x, average_step + std_dev_step, linewidth = 1, color = 'black', linestyle = 'dashed', label = 'Std Dev')
    # axs[patient_counter].plot(x, average_step - std_dev_step, linewidth = 1, color = 'black', linestyle = 'dashed')

    axs[patient_counter].set_title(f"{patient}, $N$ = {len(steps)}", fontsize = 10, y = 1.05, pad = 0); 
    axs[patient_counter].set_xticks([]); 
    axs[patient_counter].set_yticks([0, 1, 2]); 
    axs[patient_counter].set_ylim(0, 2); 

    # axs[patient_counter].set_xlabel("Time (s)")
    # axs[patient_counter].set_ylabel("Force (Body Weights)")
    patient_counter += 1; 
 
# plt.tight_layout()
plt.subplots_adjust(left = None, bottom = None, right = None, top = None, wspace = None, hspace = 0.3); 
plt.savefig('../images/array.png'); 
plt.show(); 


In [ ]:
# for patient in df: 
#     plot_average_step(df[patient], patient, 1); 

df_PD.head()

# patient = "GaPt23_01"
df_PD[patient].head()


# plot_average_step(df_PD[patient][0:1000], patient, 1); 

In [ ]:
folder_path_PD = '/Users/jacksonpowell/Documents/GitHub/learningML/data_files/PD-gait_2/'; 

all_files = glob.glob(os.path.join(folder_path_PD, "JuPt**_01.txt"))

df_PD = pd.DataFrame({1}); 

df_PD = df_process(df_PD, all_files); 



In [ ]:
rows = int(len(df_PD.columns) / 3); 
fig, axs = plt.subplots(rows, 3); 

fig.set_figheight(1.5 * rows); 
fig.set_figwidth(8); 

sampling_rate = 1; 
patient_counter = 0; 

for patient in df_PD: 

    step_indexes, num_steps = get_step_indexes(df_PD[patient], sampling_rate); 
    
    steps, average_step, std_dev_step = extract_steps(df_PD[patient], step_indexes, sampling_rate); 

    x = np.linspace(0, len(steps[0]), len(steps[0])) * 0.01; 

    axs = axs.flatten()

    counter = 0; 
    for step in steps:
        axs[patient_counter].plot(x, step, linewidth = 0.75, color = colors[counter % len(colors)]); 
        counter += 1; 
    
    axs[patient_counter].plot(x, average_step, linewidth = 1, color = 'black', label = "Average Step")
    # axs[patient_counter].plot(x, average_step + std_dev_step, linewidth = 1, color = 'black', linestyle = 'dashed', label = 'Std Dev')
    # axs[patient_counter].plot(x, average_step - std_dev_step, linewidth = 1, color = 'black', linestyle = 'dashed')

    axs[patient_counter].set_title(f"{patient}, $N$ = {len(steps)}", fontsize = 10, y = 1.05, pad = 0); 
    axs[patient_counter].set_xticks([]); 
    axs[patient_counter].set_yticks([0, 1, 2]); 
    axs[patient_counter].set_ylim(0, 2); 

    # axs[patient_counter].set_xlabel("Time (s)")
    # axs[patient_counter].set_ylabel("Force (Body Weights)")
    patient_counter += 1; 
 
# plt.tight_layout()
plt.subplots_adjust(left = None, bottom = None, right = None, top = None, wspace = None, hspace = 0.3); 
plt.savefig('../images/array.png'); 
plt.show(); 
